# 검색 증강 생성 (Retrieval Augmented Generation)

Claude는 다양한 작업에서 뛰어난 성능을 보이지만, 귀사의 고유한 비즈니스 맥락에 특화된 질문에는 어려움을 겪을 수 있습니다. 이때 **검색 증강 생성(RAG)**이 매우 유용합니다. RAG를 통해 Claude는 내부 지식 베이스나 고객 지원 문서를 활용하여 도메인 특화 질문에 대한 답변 능력을 크게 향상시킬 수 있습니다. 기업들은 고객 지원, 내부 회사 문서에 대한 Q&A, 금융 및 법률 분석 등의 워크플로우를 개선하기 위해 RAG 애플리케이션을 점점 더 많이 구축하고 있습니다.

이 가이드에서는 Claude 문서를 지식 베이스로 사용하여 RAG 시스템을 구축하고 최적화하는 방법을 보여드립니다. 다음 내용을 다룹니다:

1) 인메모리 벡터 데이터베이스와 [Voyage AI](https://www.voyageai.com/)의 임베딩을 사용한 기본 RAG 시스템 설정

2) 견고한 평가 시스템 구축. '느낌' 기반 평가를 넘어 검색 파이프라인과 엔드 투 엔드 성능을 독립적으로 측정하는 방법을 보여드립니다.

3) 요약 인덱싱과 Claude를 활용한 재순위화 등 RAG를 개선하기 위한 고급 기법 구현

일련의 개선을 통해 기본 RAG 파이프라인 대비 다음과 같은 성능 향상을 달성했습니다:

- 평균 정밀도: 0.43 --> 0.44
- 평균 재현율: 0.66 --> 0.69
- 평균 F1 점수: 0.52 --> 0.54
- 평균 MRR (Mean Reciprocal Rank): 0.74 --> 0.87
- 엔드 투 엔드 정확도: 71% --> 81%

#### 참고:

이 쿡북의 평가는 프로덕션 평가 시스템을 반영하도록 설계되었으며, 실행에 시간이 걸릴 수 있습니다. 또한 전체 평가를 실행하면 [Tier 2 이상](https://docs.claude.com/en/api/rate-limits)이 아닌 경우 속도 제한에 걸릴 수 있습니다. 토큰 사용량을 절약하려면 전체 엔드 투 엔드 평가를 건너뛰는 것을 고려하세요.

## 목차

1) 설정

2) Level 1 - 기본 RAG

3) 평가 시스템 구축

4) Level 2 - 요약 인덱싱

5) Level 3 - 요약 인덱싱 + 재순위화

## 설정

다음 라이브러리가 필요합니다:

1) `anthropic` - Claude와 상호작용

2) `voyageai` - 고품질 임베딩 생성

3) `pandas`, `numpy`, `matplotlib`, `scikit-learn` - 데이터 처리 및 시각화

[Anthropic](https://www.anthropic.com/)과 [Voyage AI](https://www.voyageai.com/)의 API 키도 필요합니다.

In [ ]:
# 필요한 패키지 설치
!pip install anthropic voyageai pandas numpy matplotlib seaborn scikit-learn

In [ ]:
import os

# 환경변수에서 API 키를 로드합니다
# 터미널에서 설정: export ANTHROPIC_API_KEY="your-key"
# 또는 .env 파일 사용을 권장합니다
# os.environ["VOYAGE_API_KEY"] = "your-voyage-api-key-here"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key-here"

In [ ]:
import os
import anthropic

# Anthropic 클라이언트 초기화
client = anthropic.Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
)

### 벡터 DB 클래스 초기화

이 예제에서는 인메모리 벡터 DB를 사용하지만, 프로덕션 애플리케이션에서는 호스팅 솔루션을 사용하는 것이 좋습니다.

In [ ]:
import json
import os
import pickle

import numpy as np
import voyageai


class VectorDB:
    """인메모리 벡터 데이터베이스 클래스"""
    
    def __init__(self, name, api_key=None):
        if api_key is None:
            api_key = os.getenv("VOYAGE_API_KEY")
        self.client = voyageai.Client(api_key=api_key)
        self.name = name
        self.embeddings = []
        self.metadata = []
        self.query_cache = {}
        self.db_path = f"./data/{name}/vector_db.pkl"

    def load_data(self, data):
        """데이터를 로드하고 임베딩을 생성합니다"""
        if self.embeddings and self.metadata:
            print("벡터 데이터베이스가 이미 로드되어 있습니다. 데이터 로딩을 건너뜁니다.")
            return
        if os.path.exists(self.db_path):
            print("디스크에서 벡터 데이터베이스를 로드합니다.")
            self.load_db()
            return

        texts = [f"Heading: {item['chunk_heading']}\n\n Chunk Text:{item['text']}" for item in data]
        self._embed_and_store(texts, data)
        self.save_db()
        print("벡터 데이터베이스가 로드되고 저장되었습니다.")

    def _embed_and_store(self, texts, data):
        """텍스트를 임베딩하고 저장합니다"""
        batch_size = 128
        result = [
            self.client.embed(texts[i : i + batch_size], model="voyage-2").embeddings
            for i in range(0, len(texts), batch_size)
        ]
        self.embeddings = [embedding for batch in result for embedding in batch]
        self.metadata = data

    def search(self, query, k=5, similarity_threshold=0.75):
        """쿼리와 유사한 문서를 검색합니다"""
        if query in self.query_cache:
            query_embedding = self.query_cache[query]
        else:
            query_embedding = self.client.embed([query], model="voyage-2").embeddings[0]
            self.query_cache[query] = query_embedding

        if not self.embeddings:
            raise ValueError("벡터 데이터베이스에 데이터가 로드되지 않았습니다.")

        similarities = np.dot(self.embeddings, query_embedding)
        top_indices = np.argsort(similarities)[::-1]
        top_examples = []

        for idx in top_indices:
            if similarities[idx] >= similarity_threshold:
                example = {
                    "metadata": self.metadata[idx],
                    "similarity": similarities[idx],
                }
                top_examples.append(example)

                if len(top_examples) >= k:
                    break
        self.save_db()
        return top_examples

    def save_db(self):
        """데이터베이스를 디스크에 저장합니다"""
        data = {
            "embeddings": self.embeddings,
            "metadata": self.metadata,
            "query_cache": json.dumps(self.query_cache),
        }
        os.makedirs(os.path.dirname(self.db_path), exist_ok=True)
        with open(self.db_path, "wb") as file:
            pickle.dump(data, file)

    def load_db(self):
        """디스크에서 데이터베이스를 로드합니다"""
        if not os.path.exists(self.db_path):
            raise ValueError(
                "벡터 데이터베이스 파일을 찾을 수 없습니다. load_data를 사용하여 새 데이터베이스를 생성하세요."
            )
        with open(self.db_path, "rb") as file:
            data = pickle.load(file)
        self.embeddings = data["embeddings"]
        self.metadata = data["metadata"]
        self.query_cache = json.loads(data["query_cache"])

## Level 1 - 기본 RAG

시작하기 위해 기본적인 RAG 파이프라인을 설정합니다. 이것은 업계에서 'Naive RAG'라고도 불립니다. 기본 RAG 파이프라인은 다음 3단계를 포함합니다:

1) 문서를 제목별로 청크 분할 - 각 소제목의 내용만 포함

2) 각 문서 임베딩

3) 코사인 유사도를 사용하여 쿼리에 답하기 위한 문서 검색

In [ ]:
import json
import logging
import xml.etree.ElementTree as ET
from collections.abc import Callable
from typing import Any

import matplotlib.pyplot as plt
from tqdm import tqdm

# 평가 데이터셋 로드
with open("evaluation/docs_evaluation_dataset.json") as f:
    eval_data = json.load(f)

# Claude 문서 로드
with open("data/anthropic_docs.json") as f:
    anthropic_docs = json.load(f)

# VectorDB 초기화
db = VectorDB("anthropic_docs")
db.load_data(anthropic_docs)


def retrieve_base(query, db):
    """기본 검색 함수: 쿼리에 대해 가장 유사한 문서를 검색합니다"""
    results = db.search(query, k=3)
    context = ""
    for result in results:
        chunk = result["metadata"]
        context += f"\n{chunk['text']}\n"
    return results, context


def answer_query_base(query, db):
    """기본 답변 함수: 검색된 문서를 기반으로 쿼리에 답변합니다"""
    documents, context = retrieve_base(query, db)
    prompt = f"""
    다음 질문에 답변하는 것이 당신의 임무입니다:
    <query>
    {query}
    </query>
    질문에 답변할 때 참고할 수 있는 다음 문서에 접근할 수 있습니다:
    <documents>
    {context}
    </documents>
    기본 맥락에 충실하고, 이미 답을 100% 확신하는 경우에만 벗어나세요.
    지금 질문에 답변하고, '답변은 다음과 같습니다' 등의 서문은 피하세요.
    """
    response = client.messages.create(
        model="claude-haiku-4-5",
        max_tokens=2500,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    return response.content[0].text

## 평가 설정

RAG 애플리케이션을 평가할 때, 검색 시스템과 엔드 투 엔드 시스템의 성능을 별도로 평가하는 것이 중요합니다.

다음을 포함하는 100개 샘플로 구성된 평가 데이터셋을 합성적으로 생성했습니다:
- 질문
- 해당 질문과 관련된 문서 청크. 질문이 들어왔을 때 검색 시스템이 검색하기를 기대하는 내용입니다.
- 질문에 대한 정답

이것은 비교적 어려운 데이터셋입니다. 일부 질문은 올바르게 답변하기 위해 둘 이상의 청크 간의 종합이 필요하므로, 시스템이 한 번에 둘 이상의 청크를 로드할 수 있어야 합니다. `evaluation/docs_evaluation_dataset.json`을 열어 데이터셋을 검사할 수 있습니다.

다음 셀을 실행하여 데이터셋 미리보기를 확인하세요.

In [ ]:
# 평가 데이터셋 미리보기
import json


def preview_json(file_path, num_items=3):
    """JSON 파일의 처음 몇 개 항목을 미리 봅니다"""
    try:
        with open(file_path) as file:
            data = json.load(file)

        if isinstance(data, list):
            preview_data = data[:num_items]
        elif isinstance(data, dict):
            preview_data = dict(list(data.items())[:num_items])
        else:
            print(f"예상치 못한 데이터 타입: {type(data)}. 미리보기할 수 없습니다.")
            return

        print(f"{file_path}의 처음 {num_items}개 항목 미리보기:")
        print(json.dumps(preview_data, indent=2, ensure_ascii=False))
        print(f"\n총 항목 수: {len(data)}")

    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
    except json.JSONDecodeError:
        print(f"유효하지 않은 JSON 파일: {file_path}")
    except Exception as e:
        print(f"오류가 발생했습니다: {str(e)}")


preview_json("evaluation/docs_evaluation_dataset.json")

# 메트릭 정의

시스템을 5가지 핵심 메트릭으로 평가합니다: 정밀도(Precision), 재현율(Recall), F1 점수, 평균 역순위(MRR), 엔드 투 엔드 정확도.

## 검색 메트릭:

### 정밀도 (Precision)
정밀도는 검색된 청크 중 실제로 관련 있는 청크의 비율을 나타냅니다. "검색한 청크 중 몇 개가 정확했는가?"라는 질문에 답합니다.

핵심 포인트:
- 높은 정밀도는 오탐(false positive)이 적은 효율적인 시스템을 나타냅니다.
- 낮은 정밀도는 많은 관련 없는 청크가 검색되고 있음을 시사합니다.
- 우리 시스템은 쿼리당 최소 3개의 청크를 검색하므로, 정밀도 점수에 영향을 미칠 수 있습니다.

공식:
$$
\text{Precision} = \frac{\text{True Positives}}{\text{Total Retrieved}} = \frac{|\text{Retrieved} \cap \text{Correct}|}{|\text{Retrieved}|}
$$

### 재현율 (Recall)
재현율은 검색 시스템의 완전성을 측정합니다. "존재하는 모든 정답 청크 중 얼마나 많이 검색했는가?"라는 질문에 답합니다.

핵심 포인트:
- 높은 재현율은 필요한 정보의 포괄적인 범위를 나타냅니다.
- 낮은 재현율은 중요한 청크가 누락되고 있음을 시사합니다.
- 재현율은 LLM이 필요한 모든 정보에 접근할 수 있도록 보장하는 데 중요합니다.

공식:
$$
\text{Recall} = \frac{\text{True Positives}}{\text{Total Correct}} = \frac{|\text{Retrieved} \cap \text{Correct}|}{|\text{Correct}|}
$$

### F1 점수
F1 점수는 정밀도와 재현율 사이의 균형 잡힌 측정을 제공합니다. 시스템 성능을 평가하기 위한 단일 메트릭이 필요할 때, 특히 불균형한 클래스 분포에서 유용합니다.

핵심 포인트:
- F1 점수는 0에서 1 사이이며, 1은 완벽한 정밀도와 재현율을 나타냅니다.
- 정밀도와 재현율의 조화 평균으로, 두 값 중 낮은 쪽으로 기울어집니다.
- 오탐과 미탐 모두 중요한 시나리오에서 유용합니다.

공식:
$$
\text{F1 Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$

### 평균 역순위 (Mean Reciprocal Rank, MRR) @k
MRR은 시스템이 관련 정보를 얼마나 잘 순위화하는지 측정합니다. 검색 결과 상단부터 시작했을 때 사용자가 원하는 것을 얼마나 빨리 찾을 수 있는지 이해하는 데 도움이 됩니다.

핵심 포인트:
- MRR은 0에서 1 사이이며, 1은 완벽(정답이 항상 첫 번째)입니다.
- 각 쿼리에 대해 첫 번째 정답 결과의 순위만 고려합니다.
- 높은 MRR은 관련 정보의 더 나은 순위화를 나타냅니다.

공식:
$$
\text{MRR} = \frac{1}{|Q|} \sum_{i=1}^{|Q|} \frac{1}{\text{rank}_i}
$$

## 엔드 투 엔드 메트릭:

### 엔드 투 엔드 정확도
LLM-as-judge (Claude 3.5 Sonnet)를 사용하여 생성된 답변이 질문과 정답을 기반으로 올바른지 평가합니다.

공식:
$$
\text{End to End Accuracy} = \frac{\text{Number of Correct Answers}}{\text{Total Number of Questions}}
$$

이 메트릭은 검색부터 답변 생성까지 전체 파이프라인을 평가합니다.

## 메트릭 계산 함수 정의

In [ ]:
def calculate_mrr(retrieved_links: list[str], correct_links: set[str]) -> float:
    """MRR(Mean Reciprocal Rank)을 계산합니다"""
    for i, link in enumerate(retrieved_links, 1):
        if link in correct_links:
            return 1 / i
    return 0


def evaluate_retrieval(
    retrieval_function: Callable, evaluation_data: list[dict[str, Any]], db: Any
) -> tuple[float, float, float, float, list[float], list[float], list[float]]:
    """검색 시스템을 평가합니다"""
    precisions = []
    recalls = []
    mrrs = []

    for i, item in enumerate(tqdm(evaluation_data, desc="검색 평가 중")):
        try:
            retrieved_chunks, _ = retrieval_function(item["question"], db)
            retrieved_links = [
                chunk["metadata"].get("chunk_link", chunk["metadata"].get("url", ""))
                for chunk in retrieved_chunks
            ]
        except Exception as e:
            logging.error(f"검색 함수 오류: {e}")
            continue

        correct_links = set(item["correct_chunks"])

        true_positives = len(set(retrieved_links) & correct_links)
        precision = true_positives / len(retrieved_links) if retrieved_links else 0
        recall = true_positives / len(correct_links) if correct_links else 0
        mrr = calculate_mrr(retrieved_links, correct_links)

        precisions.append(precision)
        recalls.append(recall)
        mrrs.append(mrr)

        if (i + 1) % 10 == 0:
            print(
                f"{i + 1}/{len(evaluation_data)} 항목 처리 완료. 현재 평균 정밀도: {sum(precisions) / len(precisions):.4f}, 평균 재현율: {sum(recalls) / len(recalls):.4f}, 평균 MRR: {sum(mrrs) / len(mrrs):.4f}"
            )

    avg_precision = sum(precisions) / len(precisions) if precisions else 0
    avg_recall = sum(recalls) / len(recalls) if recalls else 0
    avg_mrr = sum(mrrs) / len(mrrs) if mrrs else 0
    f1 = (
        2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
        if (avg_precision + avg_recall) > 0
        else 0
    )

    return avg_precision, avg_recall, avg_mrr, f1, precisions, recalls, mrrs


def evaluate_end_to_end(answer_query_function, db, eval_data):
    """엔드 투 엔드 정확도를 평가합니다"""
    correct_answers = 0
    results = []
    total_questions = len(eval_data)

    for i, item in enumerate(tqdm(eval_data, desc="엔드 투 엔드 평가 중")):
        query = item["question"]
        correct_answer = item["correct_answer"]
        generated_answer = answer_query_function(query, db)

        prompt = f"""
        당신은 Anthropic 문서에 대한 질문의 답변 정확성을 평가하는 AI 어시스턴트입니다.

        질문: {query}

        정답: {correct_answer}

        생성된 답변: {generated_answer}

        정답을 기준으로 생성된 답변이 올바른가요? 답변의 핵심에 주의를 기울이고, 사소한 세부사항의 차이는 무시하세요.

        작은 차이나 표현의 변화는 중요하지 않습니다. 생성된 답변과 정답이 본질적으로 같은 것을 말하고 있다면 올바른 것으로 표시해야 합니다.

        그러나 정답에 비해 생성된 답변에서 중요한 정보가 누락되었다면 올바르지 않은 것으로 표시해야 합니다.

        마지막으로, 정답과 생성된 답변 사이에 직접적인 모순이 있다면 생성된 답변을 올바르지 않은 것으로 간주해야 합니다.

        다음 XML 형식으로 응답하세요:
        <evaluation>
        <content>
        <explanation>설명을 여기에</explanation>
        <is_correct>true/false</is_correct>
        </content>
        </evaluation>
        """

        try:
            response = client.messages.create(
                model="claude-sonnet-4-5",
                max_tokens=1500,
                messages=[
                    {"role": "user", "content": prompt},
                    {"role": "assistant", "content": "<evaluation>"},
                ],
                temperature=0,
                stop_sequences=["</evaluation>"],
            )

            response_text = response.content[0].text
            print(response_text)
            # 신뢰할 수 있는 LLM 응답에서 XML 파싱
            evaluation = ET.fromstring(response_text)
            is_correct = evaluation.find("is_correct").text.lower() == "true"

            if is_correct:
                correct_answers += 1
            results.append(is_correct)

            logging.info(f"질문 {i + 1}/{total_questions}: {query}")
            logging.info(f"정답 여부: {is_correct}")
            logging.info("---")

        except ET.ParseError as e:
            logging.error(f"XML 파싱 오류: {e}")
            is_correct = "true" in response_text.lower()
            results.append(is_correct)
        except Exception as e:
            logging.error(f"예상치 못한 오류: {e}")
            results.append(False)

        if (i + 1) % 10 == 0:
            current_accuracy = correct_answers / (i + 1)
            print(
                f"{i + 1}/{total_questions} 질문 처리 완료. 현재 정확도: {current_accuracy:.4f}"
            )

    accuracy = correct_answers / total_questions
    return accuracy, results

## 성능 시각화 헬퍼 함수

In [ ]:
import json
import os

import matplotlib.pyplot as plt
import seaborn as sns


def plot_performance(results_folder="evaluation/json_results", include_methods=None, colors=None):
    """RAG 성능 메트릭을 시각화합니다"""
    # 기본 색상 설정
    default_colors = ["skyblue", "lightgreen", "salmon"]
    if colors is None:
        colors = default_colors

    # JSON 파일 로드
    results = []
    for filename in os.listdir(results_folder):
        if filename.endswith(".json"):
            file_path = os.path.join(results_folder, filename)
            with open(file_path) as f:
                try:
                    data = json.load(f)
                    if "name" not in data:
                        print(f"경고: {filename}에 'name' 필드가 없습니다. 건너뜁니다.")
                        continue
                    if include_methods is None or data["name"] in include_methods:
                        results.append(data)
                except json.JSONDecodeError:
                    print(f"경고: {filename}은 유효한 JSON 파일이 아닙니다. 건너뜁니다.")

    if not results:
        print("일치하는 'name' 필드가 있는 JSON 파일을 찾을 수 없습니다.")
        return

    # 데이터 검증
    required_metrics = [
        "average_precision",
        "average_recall",
        "average_f1",
        "average_mrr",
        "end_to_end_accuracy",
    ]
    for result in results.copy():
        if not all(metric in result for metric in required_metrics):
            print(f"경고: {result['name']}에 필요한 메트릭이 누락되었습니다. 건너뜁니다.")
            results.remove(result)

    if not results:
        print("검증 후 유효한 결과가 없습니다.")
        return

    # 엔드 투 엔드 정확도 기준으로 결과 정렬
    results.sort(key=lambda x: x["end_to_end_accuracy"])

    # 플롯 데이터 준비
    methods = [result["name"] for result in results]
    metrics = required_metrics

    # 플롯 설정
    plt.figure(figsize=(14, 6))
    sns.set_style("whitegrid")

    x = range(len(metrics))
    width = 0.8 / len(results)

    # 색상 팔레트 생성
    num_methods = len(methods)
    color_palette = colors[:num_methods] + sns.color_palette("husl", num_methods - len(colors))

    # 각 방법에 대한 막대 플롯
    for i, (result, color) in enumerate(zip(results, color_palette, strict=False)):
        values = [result[metric] for metric in metrics]
        offset = (i - len(results) / 2 + 0.5) * width
        bars = plt.bar([xi + offset for xi in x], values, width, label=result["name"], color=color)

        # 막대에 값 레이블 추가
        for bar in bars:
            height = bar.get_height()
            plt.text(
                bar.get_x() + bar.get_width() / 2.0,
                height,
                f"{height:.2f}",
                ha="center",
                va="bottom",
                fontsize=8,
            )

    # 플롯 커스터마이징
    plt.xlabel("메트릭", fontsize=12)
    plt.ylabel("값", fontsize=12)
    plt.title("RAG 성능 메트릭 (엔드 투 엔드 정확도 기준 정렬)", fontsize=16)
    plt.xticks(x, metrics, rotation=45, ha="right")
    plt.legend(title="방법", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.ylim(0, 1)

    plt.tight_layout()
    plt.show()

## 기본 케이스 평가

In [ ]:
import pandas as pd

# 검색 평가 실행
avg_precision, avg_recall, avg_mrr, f1, precisions, recalls, mrrs = evaluate_retrieval(
    retrieve_base, eval_data, db
)
# 엔드 투 엔드 평가 실행
e2e_accuracy, e2e_results = evaluate_end_to_end(answer_query_base, db, eval_data)

# DataFrame 생성
df = pd.DataFrame(
    {
        "question": [item["question"] for item in eval_data],
        "retrieval_precision": precisions,
        "retrieval_recall": recalls,
        "retrieval_mrr": mrrs,
        "e2e_correct": e2e_results,
    }
)

# CSV로 저장
df.to_csv("evaluation/csvs/evaluation_results_detailed.csv", index=False)
print("상세 결과가 evaluation/csvs/evaluation_results_detailed.csv에 저장되었습니다")

# 결과 출력
print(f"평균 정밀도: {avg_precision:.4f}")
print(f"평균 재현율: {avg_recall:.4f}")
print(f"평균 MRR: {avg_mrr:.4f}")
print(f"평균 F1: {f1:.4f}")
print(f"엔드 투 엔드 정확도: {e2e_accuracy:.4f}")

# 결과를 파일로 저장
with open("evaluation/json_results/evaluation_results_one.json", "w") as f:
    json.dump(
        {
            "name": "Basic RAG",
            "average_precision": avg_precision,
            "average_recall": avg_recall,
            "average_f1": f1,
            "average_mrr": avg_mrr,
            "end_to_end_accuracy": e2e_accuracy,
        },
        f,
        indent=2,
    )

print("평가 완료. 결과가 저장되었습니다.")

In [ ]:
# 성능 시각화
plot_performance("evaluation/json_results", ["Basic RAG"], colors=["skyblue"])

# Level 2: 향상된 검색을 위한 문서 요약

이 섹션에서는 문서 요약을 포함하여 검색 시스템을 개선하는 접근 방식을 구현합니다. 문서에서 직접 청크를 임베딩하는 대신, 각 청크에 대한 간결한 요약을 생성하고 이 요약을 원본 내용과 함께 임베딩 과정에서 사용합니다.

이 접근 방식은 각 문서 청크의 핵심을 더 효과적으로 캡처하여, 검색 성능을 향상시키는 것을 목표로 합니다.

이 과정의 핵심 단계:
1. 원본 문서 청크를 로드합니다.
2. 각 청크에 대해 Claude를 사용하여 2-3 문장 요약을 생성합니다.
3. 각 청크의 원본 내용과 요약을 새 JSON 파일에 저장합니다: `data/anthropic_summary_indexed_docs.json`

이 요약 강화 접근 방식은 임베딩 및 검색 단계에서 더 많은 맥락을 제공하여, 사용자 쿼리에 가장 관련 있는 문서를 이해하고 매칭하는 시스템의 능력을 향상시키도록 설계되었습니다.

## 요약 생성 및 저장

In [ ]:
import json
from tqdm import tqdm


def generate_summaries(input_file, output_file):
    """각 문서 청크에 대한 요약을 생성합니다"""
    # 원본 문서 로드
    with open(input_file) as f:
        docs = json.load(f)

    # 전체 지식 베이스에 대한 맥락 준비
    knowledge_base_context = "이것은 다양한 범용 작업에서 뛰어난 LLM인 Claude를 개발하는 프런티어 AI 연구소 Anthropic의 문서입니다. 이 문서에는 모델 세부정보와 Anthropic API 문서가 포함되어 있습니다."

    summarized_docs = []

    for doc in tqdm(docs, desc="요약 생성 중"):
        prompt = f"""
        Anthropic 문서의 다음 내용에 대한 짧은 요약을 작성하는 것이 당신의 임무입니다.

        지식 베이스 맥락:
        {knowledge_base_context}

        요약할 내용:
        제목: {doc["chunk_heading"]}
        {doc["text"]}

        위 내용을 2-3 문장으로 간략히 요약해 주세요. 요약은 핵심 포인트를 포착하고 간결해야 합니다. 이 내용에 대한 사용자 쿼리에 답변할 때 검색 파이프라인의 핵심 부분으로 사용됩니다.

        응답에 서문을 사용하지 마세요. '다음은 요약입니다' 또는 '요약은 다음과 같습니다'와 같은 문구는 금지됩니다. 요약 자체로 바로 시작하고 간결하게 작성하세요. 모든 단어가 중요합니다.
        """

        response = client.messages.create(
            model="claude-haiku-4-5",
            max_tokens=150,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )

        summary = response.content[0].text.strip()

        summarized_doc = {
            "chunk_link": doc["chunk_link"],
            "chunk_heading": doc["chunk_heading"],
            "text": doc["text"],
            "summary": summary,
        }
        summarized_docs.append(summarized_doc)

    # 요약된 문서를 새 JSON 파일로 저장
    with open(output_file, "w") as f:
        json.dump(summarized_docs, f, indent=2, ensure_ascii=False)

    print(f"요약이 생성되어 {output_file}에 저장되었습니다")


# 요약 생성 (이미 생성된 경우 주석 처리)
# generate_summaries('data/anthropic_docs.json', 'data/anthropic_summary_indexed_docs.json')

# 요약 인덱스 벡터 데이터베이스 생성

여기서는 요약 강화 문서 청크를 포함하는 새 벡터 데이터베이스를 생성합니다. 이 접근 방식은 원본 텍스트, 청크 제목, 새로 생성된 요약을 하나의 텍스트로 결합하여 임베딩합니다.

이 과정의 주요 특징:
1. Voyage AI API를 사용하여 결합된 텍스트(제목 + 요약 + 원본 내용)에 대한 임베딩을 생성합니다.
2. 임베딩과 전체 메타데이터(요약 포함)를 벡터 데이터베이스에 저장합니다.
3. 반복 쿼리의 효율성을 높이기 위해 캐싱 메커니즘을 구현합니다.
4. 데이터베이스를 디스크에 저장하여 향후 세션에서 빠르게 로드할 수 있습니다.

이 요약 인덱스 접근 방식은 더 정보가 풍부한 임베딩을 생성하여, 더 정확하고 맥락적으로 관련 있는 문서 검색을 가능하게 합니다.

In [ ]:
import json
import os
import pickle

import numpy as np
import voyageai


class SummaryIndexedVectorDB:
    """요약 인덱스를 포함한 벡터 데이터베이스 클래스"""
    
    def __init__(self, name, api_key=None):
        if api_key is None:
            api_key = os.getenv("VOYAGE_API_KEY")
        self.client = voyageai.Client(api_key=api_key)
        self.name = name
        self.embeddings = []
        self.metadata = []
        self.query_cache = {}
        self.db_path = f"./data/{name}/summary_indexed_vector_db.pkl"

    def load_data(self, data_file):
        """데이터 파일에서 데이터를 로드하고 임베딩을 생성합니다"""
        # 벡터 데이터베이스가 이미 로드되었는지 확인
        if self.embeddings and self.metadata:
            print("벡터 데이터베이스가 이미 로드되어 있습니다. 데이터 로딩을 건너뜁니다.")
            return
        # vector_db.pkl이 존재하는지 확인
        if os.path.exists(self.db_path):
            print("디스크에서 벡터 데이터베이스를 로드합니다.")
            self.load_db()
            return

        with open(data_file) as f:
            data = json.load(f)

        # 청크 제목 + 텍스트 + 요약을 함께 임베딩
        texts = [
            f"{item['chunk_heading']}\n\n{item['text']}\n\n{item['summary']}" for item in data
        ]
        
        # 128개 이상의 문서를 for 루프로 임베딩
        batch_size = 128
        result = [
            self.client.embed(texts[i : i + batch_size], model="voyage-2").embeddings
            for i in range(0, len(texts), batch_size)
        ]

        # 임베딩 평탄화
        self.embeddings = [embedding for batch in result for embedding in batch]
        self.metadata = data  # 전체 항목을 메타데이터로 저장
        self.save_db()
        print("벡터 데이터베이스가 로드되고 저장되었습니다.")

    def search(self, query, k=3, similarity_threshold=0.75):
        """쿼리와 유사한 문서를 검색합니다"""
        query_embedding = None
        if query in self.query_cache:
            query_embedding = self.query_cache[query]
        else:
            query_embedding = self.client.embed([query], model="voyage-2").embeddings[0]
            self.query_cache[query] = query_embedding

        if not self.embeddings:
            raise ValueError("벡터 데이터베이스에 데이터가 로드되지 않았습니다.")

        similarities = np.dot(self.embeddings, query_embedding)
        top_indices = np.argsort(similarities)[::-1]
        top_examples = []

        for idx in top_indices:
            if similarities[idx] >= similarity_threshold:
                example = {
                    "metadata": self.metadata[idx],
                    "similarity": similarities[idx],
                }
                top_examples.append(example)

                if len(top_examples) >= k:
                    break
        self.save_db()
        return top_examples

    def save_db(self):
        """데이터베이스를 디스크에 저장합니다"""
        data = {
            "embeddings": self.embeddings,
            "metadata": self.metadata,
            "query_cache": json.dumps(self.query_cache),
        }

        # 디렉토리가 존재하는지 확인
        os.makedirs(os.path.dirname(self.db_path), exist_ok=True)

        with open(self.db_path, "wb") as file:
            pickle.dump(data, file)

    def load_db(self):
        """디스크에서 데이터베이스를 로드합니다"""
        if not os.path.exists(self.db_path):
            raise ValueError(
                "벡터 데이터베이스 파일을 찾을 수 없습니다. load_data를 사용하여 새 데이터베이스를 생성하세요."
            )

        with open(self.db_path, "rb") as file:
            data = pickle.load(file)

        self.embeddings = data["embeddings"]
        self.metadata = data["metadata"]
        self.query_cache = json.loads(data["query_cache"])

# 요약 인덱스 임베딩을 사용한 향상된 검색

이 섹션에서는 새로운 요약 인덱스 벡터 데이터베이스를 사용하여 검색 과정을 구현합니다. 이 접근 방식은 문서 요약과 원본 내용을 포함하는 향상된 임베딩을 활용합니다.

이 업데이트된 검색 과정의 주요 측면:
1. 쿼리 임베딩을 사용하여 벡터 데이터베이스를 검색하고, 가장 유사한 상위 k개 문서를 검색합니다.
2. 검색된 각 문서에 대해 청크 제목, 요약, 전체 텍스트를 LLM에 제공하는 맥락에 포함합니다.
3. 이 풍부한 맥락을 사용하여 사용자 쿼리에 대한 답변을 생성합니다.

임베딩과 검색 단계 모두에 요약을 포함함으로써, LLM에 더 포괄적이고 집중된 맥락을 제공합니다. 이는 LLM이 각 관련 문서 청크에 대해 간결한 개요(요약)와 상세한 정보(전체 텍스트) 모두에 접근할 수 있으므로, 더 정확하고 관련 있는 답변으로 이어질 수 있습니다.

In [ ]:
def retrieve_level_two(query, db):
    """Level 2 검색: 요약 인덱스를 사용한 검색"""
    results = db.search(query, k=3)
    context = ""
    for result in results:
        chunk = result["metadata"]
        # 모델에 3가지 항목 모두 표시
        context += f"\n <document> \n {chunk['chunk_heading']}\n\n텍스트\n {chunk['text']} \n\n요약: \n {chunk['summary']} \n </document> \n"
    return results, context


def answer_query_level_two(query, db):
    """Level 2 답변: 요약 인덱스를 사용한 답변 생성"""
    documents, context = retrieve_base(query, db)
    prompt = f"""
    다음 질문에 답변하는 것이 당신의 임무입니다:
    <query>
    {query}
    </query>
    질문에 답변할 때 참고할 수 있는 다음 문서에 접근할 수 있습니다:
    <documents>
    {context}
    </documents>
    기본 맥락에 충실하고, 이미 답을 100% 확신하는 경우에만 벗어나세요.
    지금 질문에 답변하고, '답변은 다음과 같습니다' 등의 서문은 피하세요.
    """
    response = client.messages.create(
        model="claude-haiku-4-5",
        max_tokens=2500,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    return response.content[0].text

In [ ]:
# SummaryIndexedVectorDB 초기화
level_two_db = SummaryIndexedVectorDB("anthropic_docs_v2")
level_two_db.load_data("data/anthropic_summary_indexed_docs.json")

import pandas as pd

# 평가 실행
avg_precision, avg_recall, avg_mrr, f1, precisions, recalls, mrrs = evaluate_retrieval(
    retrieve_level_two, eval_data, level_two_db
)
e2e_accuracy, e2e_results = evaluate_end_to_end(answer_query_level_two, level_two_db, eval_data)

# DataFrame 생성
df = pd.DataFrame(
    {
        "question": [item["question"] for item in eval_data],
        "retrieval_precision": precisions,
        "retrieval_recall": recalls,
        "retrieval_mrr": mrrs,
        "e2e_correct": e2e_results,
    }
)

# CSV로 저장
df.to_csv("evaluation/csvs/evaluation_results_detailed_level_two.csv", index=False)
print("상세 결과가 저장되었습니다.")

# 결과 출력
print(f"평균 정밀도: {avg_precision:.4f}")
print(f"평균 재현율: {avg_recall:.4f}")
print(f"평균 MRR: {avg_mrr:.4f}")
print(f"평균 F1: {f1:.4f}")
print(f"엔드 투 엔드 정확도: {e2e_accuracy:.4f}")

# 결과를 파일로 저장
with open("evaluation/json_results/evaluation_results_level_two.json", "w") as f:
    json.dump(
        {
            "name": "Summary Indexing",
            "average_precision": avg_precision,
            "average_recall": avg_recall,
            "average_f1": f1,
            "average_mrr": avg_mrr,
            "end_to_end_accuracy": e2e_accuracy,
        },
        f,
        indent=2,
    )

print("평가 완료. 결과가 저장되었습니다.")

## 기본 RAG와 이 방법 비교 평가

In [ ]:
# 성능 시각화
plot_performance("evaluation/json_results", ["Basic RAG", "Summary Indexing"])

## Level 3 - Claude를 사용한 재순위화

검색 시스템의 마지막 개선에서, 검색된 문서의 관련성을 더욱 향상시키기 위해 재순위화 단계를 도입합니다. 이 접근 방식은 Claude의 능력을 활용하여 쿼리와 검색된 문서의 맥락과 뉘앙스를 더 잘 이해합니다.

`rerank_results` 함수는 Claude를 사용하여 초기 검색된 문서를 재평가하고 재정렬합니다:
1. 쿼리와 모든 검색된 문서의 요약을 Claude에 제시합니다.
2. Claude에게 가장 관련 있는 문서를 선택하고 순위를 매기도록 요청합니다.
3. 함수는 Claude의 응답을 파싱하여 재순위화된 문서 인덱스를 얻습니다.
4. 오류나 결과가 부족한 경우를 위한 폴백 메커니즘을 포함합니다.
5. 마지막으로, 재순위화된 결과에 내림차순 관련성 점수를 할당합니다.

`retrieve_advanced` 함수는 새로운 검색 파이프라인을 구현합니다:
1. 초기에 필요한 것보다 더 많은 문서(기본 20개, `initial_k`로 구성 가능)를 벡터 데이터베이스에서 검색합니다.
2. `rerank_results` 함수를 사용하여 이 더 큰 세트를 가장 관련 있는 문서(기본 3개, `k`로 구성 가능)로 정제합니다.
3. 마지막으로, 재순위화된 문서에서 새로운 맥락 문자열을 생성합니다.

이 과정은 초기에 더 넓은 그물을 던진 다음 AI를 사용하여 가장 적절한 정보에 집중합니다. 벡터 기반 검색과 LLM 재순위화를 결합함으로써, 이 접근 방식은 사용자 쿼리에 더 정확하고 맥락적으로 적절한 응답을 제공합니다.

평가 결과 상당한 개선을 보여줍니다:
- 정확도가 이전 시스템의 78%에서 85%로 증가했습니다.
- 재순위화를 사용하여 LLM에 표시되는 문서 수를 줄여 정밀도가 향상되었습니다.
- Claude에게 각 문서의 관련성을 순서대로 순위화하도록 요청하여 MRR(평균 역순위)이 향상되었습니다.

이러한 개선은 검색 과정에 AI 기반 재순위화를 통합하는 것의 효과를 보여줍니다.

In [ ]:
def rerank_results(query: str, results: list[dict], k: int = 5) -> list[dict]:
    """Claude를 사용하여 검색 결과를 재순위화합니다"""
    # 인덱스와 함께 요약 준비
    summaries = []
    print(len(results))

    for i, result in enumerate(results):
        summary = f"[{i}] 문서 요약: {result['metadata']['summary']}"
        summaries.append(summary)
    joined_summaries = "\n\n".join(summaries)

    prompt = f"""
    쿼리: {query}
    각각 대괄호 안의 인덱스 번호가 앞에 오는 문서 그룹이 제공됩니다. 쿼리에 답하는 데 도움이 될 가장 관련 있는 {k}개 문서만 선택하는 것이 당신의 임무입니다.

    <documents>
    {joined_summaries}
    </documents>

    관련성 순서대로 가장 관련 있는 {k}개 문서의 인덱스만 쉼표로 구분하여 XML 태그 안에 출력하세요:
    <relevant_indices>여기에 인덱스 번호를 쉼표로 구분하여 입력하세요</relevant_indices>
    """
    try:
        response = client.messages.create(
            model="claude-haiku-4-5",
            max_tokens=50,
            messages=[
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": "<relevant_indices>"},
            ],
            temperature=0,
            stop_sequences=["</relevant_indices>"],
        )

        # 응답에서 인덱스 추출
        response_text = response.content[0].text.strip()
        indices_str = response_text
        relevant_indices = []
        for idx in indices_str.split(","):
            try:
                relevant_indices.append(int(idx.strip()))
            except ValueError:
                continue  # 유효하지 않은 인덱스 건너뛰기
        print(indices_str)
        print(relevant_indices)
        
        # 유효한 인덱스가 충분하지 않으면 원래 순서의 상위 k로 폴백
        if len(relevant_indices) == 0:
            relevant_indices = list(range(min(k, len(results))))

        # 범위를 벗어난 인덱스가 없는지 확인
        relevant_indices = [idx for idx in relevant_indices if idx < len(results)]

        # 재순위화된 결과 반환
        reranked_results = [results[idx] for idx in relevant_indices[:k]]
        # 내림차순 관련성 점수 할당
        for i, result in enumerate(reranked_results):
            result["relevance_score"] = (
                100 - i
            )  # 최고 점수는 100, 각 순위마다 1씩 감소

        return reranked_results

    except Exception as e:
        print(f"재순위화 중 오류 발생: {str(e)}")
        # 재순위화 없이 상위 k 결과 반환으로 폴백
        return results[:k]


def retrieve_advanced(
    query: str, db: SummaryIndexedVectorDB, k: int = 3, initial_k: int = 20
) -> tuple[list[dict], str]:
    """고급 검색: 초기 검색 후 재순위화"""
    # 단계 1: 초기 결과 얻기
    initial_results = db.search(query, k=initial_k)

    # 단계 2: 결과 재순위화
    reranked_results = rerank_results(query, initial_results, k=k)

    # 단계 3: 재순위화된 결과에서 새 맥락 문자열 생성
    new_context = ""
    for result in reranked_results:
        chunk = result["metadata"]
        new_context += (
            f"\n <document> \n {chunk['chunk_heading']}\n\n{chunk['text']} \n </document> \n"
        )

    return reranked_results, new_context


def answer_query_advanced(query: str, db: SummaryIndexedVectorDB):
    """고급 답변: 재순위화된 문서를 사용한 답변 생성"""
    documents, context = retrieve_advanced(query, db)
    prompt = f"""
    다음 질문에 답변하는 것이 당신의 임무입니다:
    <query>
    {query}
    </query>
    질문에 답변할 때 참고할 수 있는 다음 문서에 접근할 수 있습니다:
    <documents>
    {context}
    </documents>
    기본 맥락에 충실하고, 이미 답을 100% 확신하는 경우에만 벗어나세요.
    지금 질문에 답변하고, '답변은 다음과 같습니다' 등의 서문은 피하세요.
    """
    response = client.messages.create(
        model="claude-haiku-4-5",
        max_tokens=2500,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    return response.content[0].text

## 평가

In [ ]:
# SummaryIndexedVectorDB 초기화
level_three_db = SummaryIndexedVectorDB("anthropic_docs_v3")
level_three_db.load_data("data/anthropic_summary_indexed_docs.json")

import pandas as pd

# 평가 실행
avg_precision, avg_recall, avg_mrr, f1, precisions, recalls, mrrs = evaluate_retrieval(
    retrieve_advanced, eval_data, level_three_db
)
e2e_accuracy, e2e_results = evaluate_end_to_end(answer_query_advanced, level_two_db, eval_data)

# DataFrame 생성
df = pd.DataFrame(
    {
        "question": [item["question"] for item in eval_data],
        "retrieval_precision": precisions,
        "retrieval_recall": recalls,
        "retrieval_mrr": mrrs,
        "e2e_correct": e2e_results,
    }
)

# CSV로 저장
df.to_csv("evaluation/csvs/evaluation_results_detailed_level_three.csv", index=False)
print("상세 결과가 저장되었습니다.")

# 결과 출력
print(f"평균 정밀도: {avg_precision:.4f}")
print(f"평균 재현율: {avg_recall:.4f}")
print(f"평균 F1: {f1:.4f}")
print(f"평균 MRR: {avg_mrr:.4f}")
print(f"엔드 투 엔드 정확도: {e2e_accuracy:.4f}")

# 결과를 파일로 저장
with open("evaluation/json_results/evaluation_results_level_three.json", "w") as f:
    json.dump(
        {
            "name": "Summary Indexing + Re-Ranking",
            "average_precision": avg_precision,
            "average_recall": avg_recall,
            "average_f1": f1,
            "average_mrr": avg_mrr,
            "end_to_end_accuracy": e2e_accuracy,
        },
        f,
        indent=2,
    )

print("평가 완료. 결과가 저장되었습니다.")

In [ ]:
# 성능 시각화
plot_performance(
    "evaluation/json_results",
    ["Basic RAG", "Summary Indexing", "Summary Indexing + Re-Ranking"],
    colors=["skyblue", "lightgreen", "salmon"],
)

## Promptfoo를 사용한 심화 평가

이 가이드에서는 프롬프트 엔지니어링 시 경험적 성능 측정의 중요성을 보여주었습니다. 여기서 경험적 프롬프트 엔지니어링 방법론에 대해 더 읽을 수 있습니다. Jupyter Notebook은 프롬프트 엔지니어링을 시작하기에 좋은 방법이지만, 데이터셋이 커지고 프롬프트가 많아지면 확장 가능한 도구를 활용하는 것이 중요합니다.

이 가이드의 이 섹션에서는 오픈소스 LLM 평가 툴킷인 Promptfoo를 사용하는 방법을 살펴봅니다. 시작하려면 ./evaluation 디렉토리로 이동하여 ./evaluation/README.md를 확인하세요.

Promptfoo를 사용하면 다양한 모델, 하이퍼파라미터 선택, 프롬프트를 서로 비교하는 자동화된 테스트 스위트를 매우 쉽게 구축할 수 있습니다.

예를 들어, 아래 셀을 실행하여 모든 테스트 케이스에서 Haiku와 3.5 Sonnet의 평균 성능을 확인할 수 있습니다.

In [ ]:
import json

import numpy as np
import pandas as pd

# JSON 파일 로드
with open("data/end_to_end_results.json") as f:
    data = json.load(f)

# 결과 추출
results = data["results"]["results"]

# DataFrame 생성
df = pd.DataFrame(results)

# 제공자, 프롬프트, 점수 정보 추출
df["provider"] = df["provider"].apply(lambda x: x["label"] if isinstance(x, dict) else x)
df["prompt"] = df["prompt"].apply(lambda x: x["label"] if isinstance(x, dict) else x)


# 점수를 안전하게 추출하는 함수
def extract_score(x):
    if isinstance(x, dict) and "score" in x:
        return x["score"] * 100  # 백분율로 변환
    return np.nan


df["score"] = df["gradingResult"].apply(extract_score)

# 제공자와 프롬프트별로 그룹화한 후 평균 점수 계산
result = df.groupby(["provider", "prompt"])["score"].mean().unstack()

# NaN 값을 0으로 채우기
result = result.fillna(0)

# 각 제공자의 모든 프롬프트에 대한 평균 점수 계산
result["Average"] = result.mean(axis=1)

# 평균 점수로 결과 정렬
result = result.sort_values(by="Average", ascending=False)

# 결과를 소수점 둘째 자리까지 반올림
result = result.round(2)

# 전체 통계 계산
overall_average = result["Average"].mean()
overall_std = result["Average"].std()
best_provider = result["Average"].idxmax()
worst_provider = result["Average"].idxmin()

print("\n전체 통계:")
print(f"최고 성능 제공자: {best_provider} ({result.loc[best_provider, 'Average']:.2f}%)")
print(f"최저 성능 제공자: {worst_provider} ({result.loc[worst_provider, 'Average']:.2f}%)")

## 결론

이 가이드에서는 RAG 시스템을 단계적으로 개선하는 방법을 배웠습니다:

1. **기본 RAG**: 단순한 벡터 검색으로 시작 (~71% 정확도)

2. **요약 인덱싱**: 문서 요약을 임베딩에 포함하여 검색 품질 향상

3. **재순위화**: Claude를 사용하여 검색 결과를 재정렬하여 가장 관련 있는 문서 선택 (~81% 정확도)

### 핵심 포인트:

- **경험적 평가가 중요합니다**: '느낌' 기반 평가 대신 메트릭을 사용하여 성능을 측정하세요.
- **검색과 생성을 별도로 평가하세요**: 각 구성 요소의 성능을 독립적으로 측정하여 병목 지점을 파악하세요.
- **점진적 개선이 효과적입니다**: 한 번에 하나의 개선을 적용하고 그 효과를 측정하세요.
- **도구를 활용하세요**: Promptfoo와 같은 평가 도구를 사용하여 체계적으로 테스트하세요.